<a href="https://colab.research.google.com/github/mrmarthy/DataScienceSS20/blob/mrmarthy-solution/6_b_AutoSkLearn_Regression_NY_Taxy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Block 6 Exercise 2: finding the best parameters for predicting the fare of taxi rides
We return to our Random Forest Regression and want to automatically optimize all free parameters ...

In [ ]:
import pandas as pd
import numpy as np
import folium


In [ ]:
#check if notebook runs in colab
import sys
IN_COLAB = 'google.colab' in sys.modules
print('running in Colab:',IN_COLAB)
path='..'
if IN_COLAB:
  #in colab, we need to clone the data from the repo
  !git clone https://github.com/keuperj/DataScienceSS20.git
  path='DataScienceSS20'

running in Colab: True
Cloning into 'DataScienceSS20'...
remote: Enumerating objects: 586, done.
remote: Total 586 (delta 0), reused 0 (delta 0), pack-reused 586
Receiving objects: 100% (586/586), 130.66 MiB | 11.48 MiB/s, done.
Resolving deltas: 100% (240/240), done.
Checking out files: 100% (219/219), done.


In [ ]:
# we load the data we have saved after wrangling and pre-processing in block I
X=pd.read_csv(path+'/DATA/train_cleaned.csv')
drop_columns=['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1','key','pickup_datetime','pickup_date','pickup_latitude_round3','pickup_longitude_round3','dropoff_latitude_round3','dropoff_longitude_round3']
X=X.drop(drop_columns,axis=1)
X=pd.get_dummies(X)# one hot coding
#generate labels
y=X['fare_amount']
X=X.drop(['fare_amount'],axis=1)

### Scikit Optimize
Scikit Optimize (https://scikit-optimize.github.io/stable/index.html) is a AutoML toolbox wrapped around Scikit-Learn. It allows us to use state-of-the-art automatic hyper-parameter optimization on top of our learning algorithms.   



In [ ]:
# install 
!pip install scikit-optimize

     |████████████████████████████████| 81kB 2.3MB/s 


### E 2.1 Bayesian Optimization of a Random Forest Regression Model
use Bayesian Optimization with Cross-Validation (https://scikit-optimize.github.io/stable/modules/generated/skopt.BayesSearchCV.html#skopt.BayesSearchCV) to find the best regression model. Compare
* linear regression (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression) 
* Random Forest regression (https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor)
* and SVM regression (https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR)

NOTES: this can become quite compute intensive! Hence,
* use a smaller subset of the training data to run the experiments 
* think about the range of your parameters (e.g. larger number of trees in RF or high C-values in SMV will make models expensive)
* optimize only the following parameters per model type:
    * linear: no parameters to optimize
    * RF: #trees and depth
    * SVM: C and gamma (use RBF kernel)
* parallelize -> n_jobs
* use CoLab to rum the job for up to 12h 


In [ ]:
from skopt import BayesSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

In [ ]:
#subset 1/40 -> 1/10 took way too long 
XSubSet = X.to_numpy()[:10000,:]
YSubSet = y.to_numpy()[:10000]
XTrain, XTest, YTrain, YTest = train_test_split(XSubSet, YSubSet) 

In [ ]:
linOpt = BayesSearchCV(LinearRegression(), [], n_jobs=-1, cv=5)

ValueError: ignored

In [ ]:
rfParam = {
    'max_depth': [1, 5, 10, 20, 30, 40],
    'n_estimators': [1, 5, 10, 25, 50, 100, 200, 400]
          }
rfOpt = BayesSearchCV(RandomForestRegressor(), rfParam, n_jobs=-1, cv=5)

In [ ]:
rfOpt.fit(XTrain, YTrain)

/usr/local/lib/python3.6/dist-packages/skopt/learning/gaussian_process/gpr.py:294: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
  X = check_array(X)
/usr/local/lib/python3.6/dist-packages/skopt/learning/gaussian_process/gpr.py:294: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
  X = check_array(X)
/usr/local/lib/python3.6/dist-packages/skopt/learning/gaussian_process/gpr.py:294: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended th

BayesSearchCV(cv=5, error_score='raise',
              estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                              criterion='mse', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                    

In [ ]:
rfOpt.best_score_

0.8341188103677111

In [ ]:
rfOpt.best_params_

OrderedDict([('max_depth', 10), ('n_estimators', 100)])

In [ ]:
paramDict = dict(C=[1, 5, 10, 100, 250],gamma=[250, 100, 50, 10, 2])
svmOpt = BayesSearchCV(SVR(), paramDict, n_jobs=-1, cv=5)

In [ ]:
svmOpt.fit(XTrain, YTrain)

/usr/local/lib/python3.6/dist-packages/skopt/learning/gaussian_process/gpr.py:294: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
  X = check_array(X)
/usr/local/lib/python3.6/dist-packages/skopt/learning/gaussian_process/gpr.py:294: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
  X = check_array(X)
/usr/local/lib/python3.6/dist-packages/skopt/learning/gaussian_process/gpr.py:294: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended th

BayesSearchCV(cv=5, error_score='raise',
              estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                            epsilon=0.1, gamma='scale', kernel='rbf',
                            max_iter=-1, shrinking=True, tol=0.001,
                            verbose=False),
              fit_params=None, iid=True, n_iter=50, n_jobs=-1, n_points=1,
              optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=None,
              refit=True, return_train_score=False, scoring=None,
              search_spaces={'C': [1, 5, 10, 100, 250],
                             'gamma': [250, 100, 50, 10, 2]},
              verbose=0)

In [ ]:
svmOpt.best_score_

0.0027196287665645306

In [ ]:
svmOpt.best_params_

OrderedDict([('C', 250), ('gamma', 2)])

In [23]:
linReg = LinearRegression().fit(XTrain, YTrain)

In [25]:
linReg.score(XTest, YTest)

0.6043102962804341

In [26]:
rfReg = RandomForestRegressor(max_depth=10, n_estimators=100).fit(XTrain, YTrain)

In [27]:
rfReg.score(XTest, YTest)

0.7355539377393109

In [30]:
svmReg = SVR(C=250, gamma=2).fit(XTrain, YTrain)

In [31]:
svmReg.score(XTest, YTest)

0.004867055117576235